# Actividad 4. Dirección de giro, señales de alto, autos, personas y semáforos en un video

## Documentación

## Librerias

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import time

## Variables globales

In [1]:
filePath = "videos/carretera_altos_peatones_automoviles.mp4"

## Función para la detección del giro

In [ ]:
def binarizacion(imagen):
    img = cv.cvtColor(imagen, cv.COLOR_BGR2RGB)
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img_gauss = cv.GaussianBlur(img_gray, (3, 3), 0)
    thr, img_thr = cv.threshold(img_gauss, 160, 255, cv.THRESH_BINARY)
    alto = img.shape[0]
    ancho = img.shape[1]
    ratio = 0.2
    img_r = cv.resize(img_thr, (480, 240), interpolation = cv.INTER_NEAREST)
    return(img_r)

In [ ]:
#probar la binarizacion en una imagen del video
img = cv.imread('figuras/imagen1_0.jpg')
img_bin = binarizacion(img)
fig, ax = plt.subplots(figsize = (14, 14))
ax.imshow(img_bin, cmap = 'gray')
ax.minorticks_on()
ax.grid(which = 'major', linestyle = '-', linewidth = '0.9', color = 'red')
ax.grid(which = 'minor', linestyle = ':', linewidth = '0.5', color = 'white')

In [ ]:
#Poligono de área de interés
x1 = (200, 40)
x2 = (280, 40)
x3 = (190, 50)
x4 = (290, 50)

pts_poligono = np.array([x1, x2, x3, x4], np.int32)
pts_poligono = pts_poligono.reshape((-1, 1, 2))

In [ ]:
def area_interes(imagen):
    pts1 = np.float32([x1, x2, x3, x4])
    pts2 = np.float32([[0, 0], [480, 0], [0, 240], [480, 240]])
    matrix = cv.getPerspectiveTransform(pts1, pts2)
    img_warp = cv.warpPerspective(imagen, matrix, (480, 240))
    return (img_warp)

In [ ]:
def punto_medio(imagen):
    img_cercana = imagen[220:, :]
    suma_columnas = img_cercana.sum(axis = 0)
    x_pos = np.arange(len(suma_columnas))
    mid_point = int(np.dot(x_pos, suma_columnas) / np.sum(suma_columnas))
    return mid_point

In [ ]:
imagen = 'figuras/imagen1_0.jpg'

img = cv.imread(imagen)

img_bin = binarizacion(img)
img_interes = area_interes(img_bin)
mid_point = punto_medio(img_interes)

#textos
text1 = str(0.50)
text2 = str(0.50)
text3 = 'direccion'

#agregamos el tipo de fuente
font = cv.FONT_HERSHEY_SIMPLEX

#origen de cada texto
org1 = (60, 185)
org2 = (370, 185)
org3 = (200, 100)

#tamaño 
fontScale = 0.7

#color de la fuente
color = (150, 150, 150)

#grosor
thickness = 1

#usamos la funcion 'cv.putText()' que permitirá agregar texto
cv.putText(img_interes, text1, org1, font, fontScale,
              color, thickness, cv.LINE_AA, False)
cv.putText(img_interes, text2, org2, font, fontScale,
              color, thickness, cv.LINE_AA, False)
cv.putText(img_interes, text3, org3, font, fontScale,
              color, thickness, cv.LINE_AA, False)

plt.figure(figsize = (10, 7))
cv.circle(img_interes, (mid_point, 235), 5, (100, 100, 100), -1);
plt.imshow(img_interes, cmap = 'gray')
plt.show()

In [ ]:
#Funcion suma normalizada izquierda
def sum_izquierda(imagen, valor_punto_medio):
    return np.round(np.sum(imagen[:, :valor_punto_medio].sum(axis=0))/(255*240*480), 2)

In [ ]:
#Funcion suma normalizada derecha
def sum_derecha(imagen, valor_punto_medio):
    return np.round(np.sum(imagen[:, valor_punto_medio:].sum(axis=0))/(255*240*480), 2)

In [ ]:
sum_izquierda(img_interes, mid_point)

In [ ]:
sum_derecha(img_interes, mid_point)

## Función para la detección de altos, carros y peatones

In [ ]:
#importando datos haar cascade
stop_cascade = cv.CascadeClassifier('datos/Stopsign_HAAR_19Stages.xml')
car_cascade = cv.CascadeClassifier('datos/cars.xml')
peaton_cascade = cv.CascadeClassifier('datos/haarcascade_fullbody.xml')


In [ ]:
#definiendo funcion de deteccion
def video_detect(gray, frame):
    
    altos = stop_cascade.detectMultiScale(gray, 1.12, 5)
    autos = car_cascade.detectMultiScale(gray, 1.13, 5)
    peaton = peaton_cascade.detectMultiScale(gray, 1.11, 5)

    for (x, y, w, h) in autos:

        cv.rectangle(frame, (x, y), (x + w, y + h), (213,152,221), 2)
        cv.putText(frame, 'Carro', (x, y), 6, 0.8, (213,152,221), 2)

        for (x, y, w, h) in altos:

             cv.rectangle(frame, (x, y), (x + w, y + h), (76,163,221), 2)
             cv.putText(frame, 'Alto', (x, y), 6, 0.8, (76,163,221), 2)

             for (x, y, w, h) in peaton:

                 cv.rectangle(frame, (x, y), (x + w, y + h), (13,236,175), 2)
                 cv.putText(frame, 'Peaton', (x, y), 6, 0.8, (13,236,175), 2)


        fr_gray = gray[y:y + h, x:x + w]
        fr_color = frame[y:y + h, x:x + w]
        
    return frame

## Función para la detección de semáforos

### Tratamiento de imagenes

In [ ]:
def hsv_convert(image):
    return cv.cvtColor(image, cv.COLOR_BGR2HSV)

In [ ]:
#definimos una función que retorne la mascara para el color verde
def green_finder(image):
    
    lower_green = np.array([34, 50, 232]) # 39, 50, 83
    upper_green = np.array([82, 255, 255])#91, 255, 255
    
    img_hsv = hsv_convert(image)
    mask_green = cv.inRange(img_hsv, lower_green, upper_green)
    
    return mask_green

In [ ]:
#definimos una función que retorne la mascara para el color amarillo
def yellow_finder(image):
    
    lower_green = np.array([12, 43, 232])
    upper_green = np.array([53, 255, 255])

    img_hsv = hsv_convert(image)
    mask = cv.inRange(img_hsv, lower_green, upper_green)
    mask_green = mask
    
    return mask_green

In [ ]:
#definimos una función que retorne la mascara para el color rojo
def red_finder(image):#
    lower_red1 = np.array([0,9,237])#0,65,100 older 0, 100, 100 
    upper_red1 = np.array([5,255,255]) #5,255,255
    lower_red2 = np.array([170,9,237]) #170,100,100
    upper_red2 = np.array([180,255,255]) #180,255,255
    
    img_hsv = hsv_convert(image)#
    
    mask1 = cv.inRange(img_hsv, lower_red1, upper_red1)
    mask2 = cv.inRange(img_hsv, lower_red2, upper_red2)
    
    mask_red = cv.add(mask1, mask2)
    
    return mask_red

### Circulos

In [ ]:
def circle_finder(img):
    circles = cv.HoughCircles(img, cv.HOUGH_GRADIENT, 1, 30, param1 = 50, param2 = 5, minRadius = 0, maxRadius = 30)
    return circles

In [ ]:
def print_circle(image, color_mask, label, gamma):
    
    #Estética de escritura
    font = cv.FONT_HERSHEY_SIMPLEX
    region = 5
    
    #Circulos
    circles = circle_finder(color_mask)

    #Entra si existe un circulo
    if circles is not None:
        
        circles = np.uint16(np.around(circles))
        print('In print_circle() color', label)
        
        #Para cada circulo
        for i in circles[0, :]:
            pixel_value = 0.0
            n = 0.0
            check = False
            
            for j in range(-region, region):
                for k in range(-region, region):
                    #print('region:', region)
                    #print('pixel_value:', pixel_value)
                    try:
                        pixel_value += color_mask[i[1]+j, i[0]+k]
                        
                    except:
                        print('In print_circle() data in bound')
                        
                    n += 1

            if pixel_value/n > gamma:
                print('In print_circle() passed gamma at:', pixel_value/n )
                cv.circle(image, (i[0], i[1]), i[2]+10, (0,255,0), 2)
                cv.putText(image, label, (i[0]-i[2], i[1]-i[2]-10), font, 1, (0, 255, 0), 2, cv.LINE_AA)
    
    return cv.cvtColor(image, cv.COLOR_BGR2RGB)

# Recopilación de imágenes

In [ ]:
#Captura todas las frames del video
vidcap = cv.VideoCapture(filePath)
success,image = vidcap.read()
count = 0
while success:
    cv.imwrite("all_frames_act3/stop_test_frame%d.jpg" % count, image)     # save frame as JPEG file      
    success,image = vidcap.read()
    print('Read a new frame: ', success)
    count +=1
